In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic


In [2]:
def parse_trackpoint(line):
    # Extract components from the line
    time_utc = line[1:7]  # HHMMSS
    latitude_raw = line[7:15]  # DDMMmmmN
    longitude_raw = line[15:24]  # DDDMMmmmE
    gps_altitude = int(line[25:30])  # GGGG
    pressure_altitude = int(line[30:35])  # LLLL

    # Convert UTC time
    hours = int(time_utc[:2])
    minutes = int(time_utc[2:4])
    seconds = int(time_utc[4:6])

    # Convert latitude to decimal degrees
    latitude_deg = int(latitude_raw[:2])
    latitude_min = float(latitude_raw[2:7]) / 1000
    latitude = latitude_deg + latitude_min / 60
    if latitude_raw[7] == 'S':
        latitude *= -1

    # Convert longitude to decimal degrees
    longitude_deg = int(longitude_raw[:3])
    longitude_min = float(longitude_raw[3:8]) / 1000
    longitude = longitude_deg + longitude_min / 60
    if longitude_raw[8] == 'W':
        longitude *= -1

    return {
        "time": f"{hours:02}:{minutes:02}:{seconds:02}",
        "latitude": latitude,
        "longitude": longitude,
        "gps_altitude_m": gps_altitude,
        "pressure_altitude_m": pressure_altitude,
    }


In [3]:
file_name = "data/2024-08-03 09_11_21.igc"

with open(file_name, 'r') as file:
    lines = file.readlines()

flight_data = [line for line in lines if line.startswith('B')]
df = pd.DataFrame([parse_trackpoint(line) for line in flight_data])
del lines, flight_data
df.head()

,time,latitude,longitude,gps_altitude_m,pressure_altitude_m
0,09:11:21,40.031450,32.32855,1142,1147
1,09:11:22,40.031450,32.32855,1142,1147
2,09:11:23,40.031450,32.32855,1142,1147
3,09:11:24,40.031450,32.32855,1142,1147
4,09:11:25,40.031433,32.32855,1142,1147


In [4]:
for i in range(1, len(df)-1):
    if i % 10 > 0:
        df = df.drop([i])

flight_loginterval = int(df["time"].iloc[1][-2:]) - \
    int(df["time"].iloc[0][-2:])


def calculate_distance(row):
    return geodesic((row["previous_latitude"], row["previous_longitude"]), (row["latitude"], row["longitude"])).meters


prev_total_distance = 0


def caluculate_total_distance(row):
    global prev_total_distance
    prev_total_distance += row["distance_m"]
    return prev_total_distance

In [5]:

df["previous_latitude"] = df["latitude"].shift(1)
df["previous_longitude"] = df["longitude"].shift(1)
df["time"] = pd.to_datetime(df["time"])
df["elapsed_time"] = (df["time"] - df["time"].iloc[0]).dt.total_seconds()
df.fillna(0, inplace=True)
df["distance_m"] = df.apply(calculate_distance, axis=1)
df["total_distance_m"] = df.apply(
    lambda row: caluculate_total_distance(row), axis=1)
df["climb_m"] = df["gps_altitude_m"].diff()
df["climb_rate_m/s"] = df["climb_m"] / flight_loginterval
df["speed_km/s"] = (df["distance_m"]/1000) / (flight_loginterval/3600)
df["distance_from_start_m"] = df.apply(lambda row: geodesic(
    (df["latitude"].iloc[0], df["longitude"].iloc[0]), (row["previous_latitude"], row["previous_longitude"])).meters, axis=1)
df.to_csv("flight_data.csv", index=False)
df.head()

C:\Users\serka\AppData\Local\Temp\ipykernel_35880\254540318.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["time"] = pd.to_datetime(df["time"])


,time,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,elapsed_time,distance_m,total_distance_m,climb_m,climb_rate_m/s,speed_km/s,distance_from_start_m
0,2025-03-30 09:11:21,40.031450,32.328550,1142,1147,0.000000,0.000000,0.0,5.512940e+06,5.512940e+06,NaN,NaN,1.984659e+06,5.512940e+06
10,2025-03-30 09:11:31,40.031450,32.328550,1142,1149,40.031450,32.328550,10.0,0.000000e+00,5.512940e+06,0.0,0.0,0.000000e+00,0.000000e+00
20,2025-03-30 09:11:41,40.031567,32.328267,1137,1140,40.031450,32.328550,20.0,2.743475e+01,5.512968e+06,-5.0,-0.5,9.876508e+00,0.000000e+00
30,2025-03-30 09:11:51,40.031900,32.327767,1147,1149,40.031567,32.328267,30.0,5.649079e+01,5.513024e+06,10.0,1.0,2.033668e+01,2.743475e+01
40,2025-03-30 09:12:01,40.032700,32.328383,1139,1141,40.031900,32.327767,40.0,1.032515e+02,5.513128e+06,-8.0,-0.8,3.717053e+01,8.346839e+01
